In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import time
import pickle
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.sports-reference.com/cbb/schools/abilene-christian/2018.html#per_poss::none'
team_names_filepath = 'sos/sos_list2018.csv'

In [3]:
df = pd.read_html(url)[2]

In [4]:
df

,Rk,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Jalone Friday,32,28,22.3,5.0,11.0,0.456,3.6,6.8,...,0.702,1.2,3.9,5.1,0.5,0.3,1.6,1.7,3.5,13.3
1,2,Jaren Lewis,32,32,31.2,5.0,9.1,0.546,4.6,8.0,...,0.791,2.5,5.3,7.7,1.8,1.6,0.9,2.3,2.4,13.2
2,3,Tevin Foster,32,30,32.4,3.8,9.1,0.419,2.4,4.6,...,0.802,0.6,2.5,3.1,3.5,1.5,0.1,2.6,2.3,12.3
3,4,Jaylen Franklin,32,32,32.1,4.4,8.0,0.557,4.3,7.3,...,0.539,1.1,2.8,3.9,4.5,1.8,0.2,2.7,2.4,10.5
4,5,Payten Ricks,26,17,22.5,2.0,5.3,0.387,1.0,2.6,...,0.745,0.3,1.5,1.8,1.7,0.8,0.0,1.3,1.7,6.6
5,6,Drake Green,32,8,16.4,1.5,3.5,0.423,0.5,0.8,...,0.857,0.1,1.1,1.2,0.9,0.5,0.1,0.8,1.9,4.1
6,7,Hayden Howell,32,3,10.2,1.6,3.0,0.526,1.4,2.6,...,0.600,0.7,1.3,2.0,0.3,0.2,0.1,0.8,1.8,3.8
7,8,Hayden Farquhar,31,1,9.7,1.4,3.6,0.378,0.8,1.3,...,0.588,0.5,2.0,2.5,0.4,0.4,0.3,0.6,1.3,3.5
8,9,B.J. Maxwell,31,2,9.9,1.4,3.0,0.467,1.0,2.0,...,0.563,0.9,1.4,2.2,0.6,0.3,0.4,0.5,1.7,3.4
9,10,Isaiah Tripp,27,5,12.0,1.0,2.6,0.371,0.5,1.2,...,0.688,0.1,0.9,1.0,0.9,0.6,0.0,0.6,0.9,2.8


In [5]:
def team_list(filepath):
    '''
    Create dictionary of school names and formatted school names for mapping
    '''
    team_names = pd.read_csv(filepath)
    school_list = team_names['school_format'].tolist()
    return school_list

In [6]:
def player_df_creator(teams, seasons):
    '''
    Inputs:
        team = team (formatted as in url)
        season = season year

    Output: DataFrame of all games
    '''
    player_df = pd.DataFrame()

    for season in seasons:

        for team in teams:
        
            url = 'https://www.sports-reference.com/cbb/schools/{}/{}.html#per_poss::none'.format(team, season)

            '''Read team gamelog'''
            df = pd.read_html(url)[2]
            
            '''Drop Rk col'''
            df = df.drop(['Rk'], axis=1)

            '''Add Team col'''
            df['Team'] = team
            
            '''Add df to games_df'''
            player_df = player_df.append(df, ignore_index=True)

        time.sleep(15)

    return player_df

In [7]:
teams = ['michigan-state', 'michigan'] #team_list(team_names_filepath)
seasons = [2018]
player_df = player_df_creator(teams, seasons)

In [8]:
player_df.describe()

,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,27.000000,29.000000,...,26.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,28.482759,13.103448,14.951724,2.006897,4.206897,0.455207,1.386207,2.520690,0.554926,0.617241,...,0.601654,0.665517,1.944828,2.606897,1.210345,0.403448,0.386207,0.800000,1.282759,5.575862
std,13.518825,16.395437,10.397926,1.866228,3.820334,0.166895,1.393024,2.493617,0.117032,0.736337,...,0.270918,0.629215,1.619653,2.118950,1.503887,0.322360,0.590837,0.685565,0.863162,5.288306
min,1.000000,0.000000,1.600000,0.000000,0.500000,0.000000,0.000000,0.000000,0.409000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,5.200000,0.600000,1.000000,0.412000,0.400000,0.800000,0.480000,0.000000,...,0.548250,0.200000,0.800000,1.000000,0.300000,0.200000,0.100000,0.300000,0.500000,1.500000
50%,35.000000,1.000000,12.500000,1.100000,2.100000,0.467000,0.800000,1.400000,0.527000,0.200000,...,0.628500,0.400000,1.400000,2.300000,0.800000,0.300000,0.200000,0.600000,1.100000,2.900000
75%,38.000000,33.000000,25.800000,3.400000,6.800000,0.528000,2.200000,3.900000,0.595500,1.100000,...,0.819500,1.000000,2.800000,3.400000,1.300000,0.600000,0.500000,1.400000,1.700000,10.900000
max,41.000000,41.000000,34.900000,6.100000,13.400000,0.706000,4.600000,7.700000,1.000000,2.100000,...,1.000000,2.700000,5.700000,7.100000,6.900000,1.300000,3.000000,2.500000,3.200000,17.100000


In [9]:
player_df

,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team
0,Miles Bridges,34,33,31.4,6.1,13.4,0.457,4.1,7.7,0.527,...,1.3,5.7,7.0,2.7,0.6,0.8,2.0,1.6,17.1,michigan-state
1,Cassius Winston,35,34,28.1,4.1,8.1,0.507,1.9,3.7,0.519,...,0.2,3.2,3.4,6.9,0.7,0.1,2.5,2.0,12.6,michigan-state
2,Nick Ward,35,34,18.9,4.6,7.1,0.648,4.6,7.1,0.647,...,2.7,4.4,7.1,0.5,0.3,1.3,1.7,2.3,12.4,michigan-state
3,Joshua Langford,35,35,27.1,4.3,10.1,0.429,3.1,7.0,0.441,...,0.8,2.2,3.0,1.5,0.4,0.1,1.5,2.4,11.7,michigan-state
4,Jaren Jackson Jr.,35,34,21.8,3.4,6.6,0.513,2.3,3.9,0.596,...,1.5,4.3,5.8,1.1,0.6,3.0,1.8,3.2,10.9,michigan-state
5,Matt McQuaid,35,2,20.5,1.9,4.7,0.406,0.4,0.9,0.469,...,0.7,1.2,1.9,1.3,0.3,0.1,0.7,1.7,6.0,michigan-state
6,Gavin Schilling,34,1,9.7,1.1,1.5,0.706,1.1,1.5,0.706,...,1.2,2.2,3.4,0.4,0.2,0.4,0.6,1.5,2.9,michigan-state
7,Xavier Tillman,35,0,8.7,1.1,1.7,0.650,1.1,1.7,0.650,...,1.0,1.6,2.6,0.3,0.3,0.7,0.5,1.0,2.8,michigan-state
8,Kenny Goins,31,0,13.9,0.9,1.9,0.458,0.7,1.4,0.523,...,0.8,2.0,2.8,1.2,0.4,0.7,0.5,1.1,2.1,michigan-state
9,Lourawls Nairn,35,1,16.7,0.8,1.5,0.500,0.6,1.1,0.579,...,0.0,0.9,0.9,2.9,0.2,0.1,0.8,1.0,1.7,michigan-state


In [10]:
player_df[player_df['MP'] > np.percentile(player_df['MP'], 50)]

,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team
0,Miles Bridges,34,33,31.4,6.1,13.4,0.457,4.1,7.7,0.527,...,1.3,5.7,7.0,2.7,0.6,0.8,2.0,1.6,17.1,michigan-state
1,Cassius Winston,35,34,28.1,4.1,8.1,0.507,1.9,3.7,0.519,...,0.2,3.2,3.4,6.9,0.7,0.1,2.5,2.0,12.6,michigan-state
2,Nick Ward,35,34,18.9,4.6,7.1,0.648,4.6,7.1,0.647,...,2.7,4.4,7.1,0.5,0.3,1.3,1.7,2.3,12.4,michigan-state
3,Joshua Langford,35,35,27.1,4.3,10.1,0.429,3.1,7.0,0.441,...,0.8,2.2,3.0,1.5,0.4,0.1,1.5,2.4,11.7,michigan-state
4,Jaren Jackson Jr.,35,34,21.8,3.4,6.6,0.513,2.3,3.9,0.596,...,1.5,4.3,5.8,1.1,0.6,3.0,1.8,3.2,10.9,michigan-state
5,Matt McQuaid,35,2,20.5,1.9,4.7,0.406,0.4,0.9,0.469,...,0.7,1.2,1.9,1.3,0.3,0.1,0.7,1.7,6.0,michigan-state
8,Kenny Goins,31,0,13.9,0.9,1.9,0.458,0.7,1.4,0.523,...,0.8,2.0,2.8,1.2,0.4,0.7,0.5,1.1,2.1,michigan-state
9,Lourawls Nairn,35,1,16.7,0.8,1.5,0.500,0.6,1.1,0.579,...,0.0,0.9,0.9,2.9,0.2,0.1,0.8,1.0,1.7,michigan-state
12,Moritz Wagner,39,39,27.6,5.5,10.5,0.528,3.9,6.4,0.614,...,1.5,5.7,7.1,0.8,1.0,0.5,1.4,3.1,14.6,michigan
13,Charles Matthews,41,41,30.1,5.0,10.1,0.495,4.2,7.5,0.557,...,1.3,4.2,5.5,2.4,0.7,0.6,2.0,2.1,13.0,michigan


In [11]:
ns_df = pd.read_csv('../players/stns2018.csv', index_col=0)

In [12]:
ns_df.head()

,3PM-A,A,BLK,DATE,DEF,FGM-A,FTM-A,MATCH,MIN,OFF,PF,PLAYER,PTS,STL,TEAM,TO,TOT
0,1-2,2,1,2017-11-10,2.0,1-2,0-0,Savannah State\n vs. Cincinnati\n,22:00,0.0,0.0,Javaris Jenkins,3.0,0.0,Savannah State\n,2.0,2.0
1,0-3,1,0,2017-11-10,3.0,6-12,1-2,Savannah State\n vs. Cincinnati\n,20:00,2.0,3.0,Jahlin Smith,13.0,1.0,Savannah State\n,4.0,5.0
2,1-9,5,0,2017-11-10,3.0,3-14,4-6,Savannah State\n vs. Cincinnati\n,30:00,1.0,2.0,Ty'Lik Evans,11.0,3.0,Savannah State\n,2.0,4.0
3,1-4,2,0,2017-11-10,2.0,3-11,4-5,Savannah State\n vs. Cincinnati\n,25:00,1.0,2.0,Dexter McClanahan,11.0,2.0,Savannah State\n,2.0,3.0
4,4-7,6,0,2017-11-10,5.0,5-9,0-0,Savannah State\n vs. Cincinnati\n,26:00,0.0,4.0,Austin Dasent,14.0,3.0,Savannah State\n,5.0,5.0


Beautiful Soup Test Run

In [37]:
def player_stats_scraper(teams, seasons):
    '''
    Inputs:
        team = team (formatted as in url)
        season = season year

    Output: DataFrame of all games
    '''
    player_per100_df = pd.DataFrame()

    for season in seasons:

        for team in teams:
        
            url = 'https://www.sports-reference.com/cbb/schools/{}/{}.html#per_poss::none'.format(team, season)
            
            # Extract html from player page
            req = requests.get(url).text
            
            # Create soup object form html
            soup = BeautifulSoup(req, 'html.parser')
            
            # Extract placeholder classes
            placeholders = soup.find_all('div', {'class': 'placeholder'})
            for x in placeholders:
                # Get elements after placeholder and combine into one string
                comment = ''.join(x.next_siblings)
                # Parse comment back into soup object
                soup_comment = BeautifulSoup(comment, 'html.parser')
                # Extract correct table from soup object using 'id' attribute
                tables = soup_comment.find_all('table', attrs={"id":"per_poss"})
                # Iterate tables
                for tag in tables:  #change this shit til it works
                    # Turn table from html to pandas DataFrame
                    df = pd.read_html(tag.prettify())[0]
                    # Extract a player's stats from their most recent college season
                    table = df.iloc[:, :]
                    # Add Team Column
                    table['Team'] = team
                    # Add individual player stats to full per_poss DataFrame
                    player_per100_df = player_per100_df.append(table).reset_index()
                    # Filter out irrelevant columns
                    player_per100_df = player_per100_df[['Player', 'G', 'GS', 'MP',
                    'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT',
                    'FTA', 'FT%', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                    'ORtg', 'DRtg', 'Team']]
                    
                    
        time.sleep(15)

    return player_per100_df

In [38]:
per100 = player_stats_scraper(teams, seasons)

In [39]:
per100

,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Team
0,Nick Ward,35,34,662,14.4,22.2,0.648,14.3,22.1,0.647,...,22.1,1.6,1.0,4.2,5.1,7.3,38.6,121.7,90.2,michigan-state
1,Miles Bridges,34,33,1067,11.5,25.1,0.457,7.6,14.4,0.527,...,13.0,5.1,1.2,1.4,3.7,2.9,32.0,118.2,95.5,michigan-state
2,Jaren Jackson Jr.,35,34,764,9.1,17.8,0.513,6.2,10.4,0.596,...,15.6,3.0,1.6,8.1,4.8,8.6,29.3,120.6,86.4,michigan-state
3,Cassius Winston,35,34,985,8.5,16.8,0.507,4.1,7.8,0.519,...,7.0,14.4,1.4,0.1,5.2,4.1,26.3,131.2,99.8,michigan-state
4,Joshua Langford,35,35,949,9.4,21.9,0.429,6.7,15.1,0.441,...,6.5,3.3,0.8,0.2,3.2,5.1,25.3,109.0,102.6,michigan-state
5,Xavier Tillman,35,0,305,7.5,11.5,0.650,7.5,11.5,0.650,...,17.7,1.9,1.7,4.4,3.1,6.7,19.0,125.8,90.6,michigan-state
6,Gavin Schilling,34,1,329,6.4,9.1,0.706,6.4,9.1,0.706,...,20.7,2.1,1.4,2.1,3.9,9.1,17.8,121.0,91.3,michigan-state
7,Matt McQuaid,35,2,718,5.5,13.5,0.406,1.2,2.6,0.469,...,5.6,3.7,0.9,0.2,2.0,4.8,17.2,123.6,103.5,michigan-state
8,Kenny Goins,31,0,431,3.7,8.0,0.458,3.1,6.0,0.523,...,12.0,5.0,1.6,3.0,2.2,4.8,8.8,115.2,94.5,michigan-state
9,Conner George,16,0,54,3.3,16.3,0.200,3.3,7.6,0.429,...,19.6,0.0,3.3,0.0,2.2,6.5,6.5,64.2,90.8,michigan-state


In [40]:
per100[per100['MP'] > np.percentile(per100['MP'], 50)]

,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Team
0,Nick Ward,35,34,662,14.4,22.2,0.648,14.3,22.1,0.647,...,22.1,1.6,1.0,4.2,5.1,7.3,38.6,121.7,90.2,michigan-state
1,Miles Bridges,34,33,1067,11.5,25.1,0.457,7.6,14.4,0.527,...,13.0,5.1,1.2,1.4,3.7,2.9,32.0,118.2,95.5,michigan-state
2,Jaren Jackson Jr.,35,34,764,9.1,17.8,0.513,6.2,10.4,0.596,...,15.6,3.0,1.6,8.1,4.8,8.6,29.3,120.6,86.4,michigan-state
3,Cassius Winston,35,34,985,8.5,16.8,0.507,4.1,7.8,0.519,...,7.0,14.4,1.4,0.1,5.2,4.1,26.3,131.2,99.8,michigan-state
4,Joshua Langford,35,35,949,9.4,21.9,0.429,6.7,15.1,0.441,...,6.5,3.3,0.8,0.2,3.2,5.1,25.3,109.0,102.6,michigan-state
7,Matt McQuaid,35,2,718,5.5,13.5,0.406,1.2,2.6,0.469,...,5.6,3.7,0.9,0.2,2.0,4.8,17.2,123.6,103.5,michigan-state
10,Lourawls Nairn,35,1,583,2.7,5.4,0.500,2.2,3.8,0.579,...,3.2,10.3,0.8,0.2,2.9,3.4,6.0,113.6,104.1,michigan-state
14,Moritz Wagner,39,39,1078,12.2,23.0,0.528,8.6,14.0,0.614,...,15.7,1.9,2.1,1.1,3.2,6.8,32.1,117.3,92.3,michigan
16,Charles Matthews,41,41,1234,10.1,20.4,0.495,8.4,15.1,0.557,...,11.1,4.8,1.5,1.3,4.1,4.3,26.1,105.7,96.5,michigan
19,Muhammad-Ali Abdur-Rahkman,41,41,1432,7.6,17.6,0.431,4.5,9.3,0.482,...,6.7,5.6,1.5,0.3,1.3,2.7,22.4,122.1,99.7,michigan
